# import libraries

In [254]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



# read in data

In [255]:
df = pd.read_csv('HR_comma_sep.csv')

In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   sales                  14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [257]:
df['Department'] = df['sales']
df = df.drop(columns=['sales'])

In [258]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,Department
0,0.38,0.53,2,157,3,0,1,0,low,sales
1,0.80,0.86,5,262,6,0,1,0,medium,sales
2,0.11,0.88,7,272,4,0,1,0,medium,sales
3,0.72,0.87,5,223,5,0,1,0,low,sales
4,0.37,0.52,2,159,3,0,1,0,low,sales


data will need to be scaled (use minmax/robust if not normally distributed and standard if normally distributed)

no null values

salary and sales (department) will need to be encoded

left is the predictor


## EDA

In [259]:
fig = px.imshow(df.corr(numeric_only=True), text_auto=True, aspect="auto", color_continuous_scale='agsunset', title='Correlation Matrix',width=800, height=800)
fig.show()

weak correlations: neg corr between left and satisfacton, pos corr between num projects and monthly hors, pos corr between last eval and monthly hours, pos corr between num hours and last eval

In [260]:
fig = px.histogram(df, x="satisfaction_level", marginal='box', title='Satisfaction Level Distribution', color_discrete_sequence=['indigo'])
fig.show()

not normally distributed

most values between 0.44 and 0.82

In [261]:
fig = px.histogram(df, x="last_evaluation", marginal='box', title='Evaluation Rating Distribution', color_discrete_sequence=['lightcoral'])
fig.show()

not normally distributed

most values between 0.56 and 0.87

there are two peaks around 0.55 and 0.87

In [262]:
fig = px.histogram(df, x="average_montly_hours", marginal='box', title='Average Monthly Hours in Office Distribution', color_discrete_sequence=['mediumvioletred'])
fig.show()

not normally distributed

most values between 156 and 245

two peaks around 145 and 255

In [263]:
fig = px.histogram(df, x="time_spend_company", marginal='box', title='Number of Years with Company Distribution', color_discrete_sequence=['peachpuff'])
fig.show()

left skewed

most employees have been with the company 3 years with the range for most employees being 2 and five years

outliers for values greater than 5 years

In [264]:
fig = px.histogram(df, x="number_project", marginal='box', title='Number of Projects Distribution', color_discrete_sequence=['hotpink'])
fig.show()

left skewed

most employees have 4 projects

2 - 7 projects for employees is the range

In [265]:
fig = px.histogram(df, x="promotion_last_5years", marginal='box', title='Number of Promotions in Last 5 years Distribution', color_discrete_sequence=['darkorchid'])
fig.show()

very few employees received a rasie in the the last 5 years

In [266]:
project_left_counts = df.groupby(['number_project', 'left']).size().reset_index(name='count')
fig = px.histogram(project_left_counts, x='number_project', y='count', color='left', title='Number of Projects by Employee Turnover', barmode='group', color_discrete_sequence=['lightseagreen', 'salmon'], nbins=7)
fig.show()

2, 6, 7 projects - higher turnover than retention

In [267]:
project_left_counts = df.groupby(['number_project', 'left']).size().reset_index(name='count')
fig = px.histogram(project_left_counts, x='number_project', y='count', color='left', title='Number of Projects by Employee Turnover', barmode='group', color_discrete_sequence=['lightseagreen', 'salmon'])
fig.show()

turnover greater than retention if the number of projects an employee has is between 6-7

between 4-5 projects has less turnover

In [268]:
salary_counts = df.groupby(['salary', 'left']).size().reset_index(name='count')
fig = px.histogram(salary_counts, x='salary', y='count', color='left', title='Salary Level by Employee Turnover', barmode='group', color_discrete_sequence=['mediumslateblue', 'tomato'])
fig.show()

higher turnover when salary low and medium

In [269]:
accident_counts = df.groupby(['Work_accident', 'left']).size().reset_index(name='count')
fig = px.histogram(accident_counts, x='Work_accident', y='count', color='left', title='Work Accident by Employee Turnover', barmode='group', color_discrete_sequence=['indigo', 'coral'])
fig.show()

no work accident but more turnover - i think this is more of a volume thing

## Classification Modeling

encoding

In [270]:
df.select_dtypes('O').nunique()

salary         3
Department    10
dtype: int64

In [271]:
df = pd.get_dummies(df, columns=['salary','Department'], drop_first=True)

In [272]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary_low,salary_medium,Department_RandD,Department_accounting,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical
0,0.38,0.53,2,157,3,0,1,0,True,False,False,False,False,False,False,False,True,False,False
1,0.80,0.86,5,262,6,0,1,0,False,True,False,False,False,False,False,False,True,False,False
2,0.11,0.88,7,272,4,0,1,0,False,True,False,False,False,False,False,False,True,False,False
3,0.72,0.87,5,223,5,0,1,0,True,False,False,False,False,False,False,False,True,False,False
4,0.37,0.52,2,159,3,0,1,0,True,False,False,False,False,False,False,False,True,False,False


train test split

In [273]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['left'], axis=1), df['left'], test_size=0.2)

In [274]:
X_train.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_low,salary_medium,Department_RandD,Department_accounting,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical
11335,0.83,0.50,6,165,3,0,0,False,True,False,False,False,True,False,False,False,False,False
13391,0.92,0.93,4,225,2,0,1,False,True,True,False,False,False,False,False,False,False,False
10448,0.94,0.89,4,192,2,0,0,True,False,False,False,False,False,False,True,False,False,False
6632,0.95,0.74,3,139,2,0,0,False,True,False,False,False,False,False,False,False,False,False
7515,0.94,0.77,5,233,4,0,0,False,False,False,False,False,False,False,False,False,False,True


In [275]:
X_test.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_low,salary_medium,Department_RandD,Department_accounting,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical
6747,0.80,0.95,3,247,3,0,0,False,True,False,False,True,False,False,False,False,False,False
6492,0.64,0.57,4,217,3,0,0,False,False,False,False,False,False,True,False,False,False,False
4546,0.71,0.79,4,133,3,0,1,False,True,False,False,False,False,False,False,False,False,True
5115,0.78,0.45,3,253,6,0,1,False,True,False,False,False,False,False,False,False,True,False
7087,0.94,0.80,5,170,4,0,0,False,False,False,False,False,False,False,False,False,False,False


scaling

In [276]:
mm = MinMaxScaler()

X_train_transformed = pd.DataFrame(mm.fit_transform(X_train), columns=X_train.columns)  

X_test_transformed = pd.DataFrame(mm.transform(X_test), columns=X_test.columns)

In [277]:
X_train_transformed.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_low,salary_medium,Department_RandD,Department_accounting,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical
0,0.8131868132,0.218750,0.8,0.3224299065,0.125,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.9120879121,0.890625,0.4,0.6028037383,0.000,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.9340659341,0.828125,0.4,0.4485981308,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.9450549451,0.593750,0.2,0.2009345794,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.9340659341,0.640625,0.6,0.6401869159,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [278]:
X_test_transformed.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_low,salary_medium,Department_RandD,Department_accounting,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical
0,0.7802197802,0.921875,0.2,0.7056074766,0.125,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.6043956044,0.328125,0.4,0.5654205607,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.6813186813,0.671875,0.4,0.1728971963,0.125,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.7582417582,0.140625,0.2,0.7336448598,0.500,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.9340659341,0.687500,0.6,0.3457943925,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


smote

In [279]:
y_train.value_counts()

left
0    9144
1    2855
Name: count, dtype: int64

In [280]:
fig = px.histogram(y_train, nbins=2, title='Distribution of (Employee Turnover)', color=y_train.map({0: 'Stayed', 1: 'Left'}), color_discrete_sequence=['teal', 'salmon'])
fig.update_xaxes(title='Left (0 = Stayed, 1 = Left)')
fig.update_yaxes(title='Count')
fig.show()

In [281]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train_transformed, y_train)

In [282]:
fig = px.histogram(
	y_train_smote, 
	nbins=2, 
	title='Distribution of (Employee Turnover)', 
	color=y_train_smote.map({0: 'Stayed', 1: 'Left'}),
	color_discrete_sequence=['teal', 'salmon']
)
fig.update_xaxes(title='Left (0 = Stayed, 1 = Left)')
fig.update_yaxes(title='Count')
fig.show()

In [283]:
knn = KNeighborsClassifier(n_neighbors=5)

In [284]:
knn.fit(X_train_smote, y_train_smote)

KNeighborsClassifier()

Train Results

In [285]:
y_pred_train = knn.predict(X_train_smote)

In [286]:
y_pred_train

array([0, 0, 0, ..., 1, 1, 1])

In [287]:
print(classification_report(y_train_smote, y_pred_train))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97      9144
           1       0.95      0.99      0.97      9144

    accuracy                           0.97     18288
   macro avg       0.97      0.97      0.97     18288
weighted avg       0.97      0.97      0.97     18288



Test Results

In [288]:
y_pred = knn.predict(X_test_transformed)

In [289]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [290]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95      2284
           1       0.81      0.94      0.87       716

    accuracy                           0.93      3000
   macro avg       0.89      0.93      0.91      3000
weighted avg       0.94      0.93      0.93      3000



stratified k fold

In [291]:
skf = StratifiedKFold(n_splits=5, shuffle=True)    

logistic regression

In [292]:
lr = LogisticRegression()

In [293]:
lr_kfold_scores = cross_val_score(lr, X_train_smote, y_train_smote, cv=skf, scoring='accuracy') 

In [294]:
np.mean(np.abs(lr_kfold_scores))

np.float64(0.7754805339729839)

In [295]:
lr.fit(X_train_smote, y_train_smote)
y_pred_lr = lr.predict(X_test_transformed)
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.75      0.83      2284
           1       0.50      0.80      0.62       716

    accuracy                           0.76      3000
   macro avg       0.71      0.78      0.72      3000
weighted avg       0.82      0.76      0.78      3000



In [296]:
y_score_lr = lr.predict_proba(X_test_transformed)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_score_lr)
roc_auc = auc(fpr, tpr)

In [297]:
fig_hist = px.histogram(x=y_score_lr, nbins=50, title='Predicted Probabilities of Employee Turnover (Logistic Regression)', color = y_test.map({0: 'Stayed', 1: 'Left'}), labels={'x': 'Predicted Probability', 'color': 'Actual Turnover'})
fig_hist.show()

In [298]:
# tpr

In [299]:
# len(fpr), len(tpr), len(thresholds)

In [300]:
df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr
}, index=thresholds)
df.index.name = "Thresholds"
df.columns.name = "Rate"

fig_thresh = px.line(
    df, title='TPR and FPR at every threshold',
    width=700, height=500, render_mode='SVG'
)

fig_thresh.update_yaxes(scaleanchor="x", scaleratio=1)
fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
fig_thresh.show()

note: if you have over a certain amount of points, the lines may not show due to webgl limitations...use render_mode='SVG' to fix

source: https://community.plotly.com/t/plotly-express-line-charts-are-not-shown/39715

random forest

In [301]:
rf = RandomForestClassifier()

In [302]:
rf_kfold_scores = cross_val_score(rf, X_train_smote, y_train_smote, cv=skf, scoring='accuracy') 

In [303]:
rf.fit(X_train_smote, y_train_smote)
y_pred_rf = rf.predict(X_test_transformed)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2284
           1       1.00      0.96      0.98       716

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.99      3000
weighted avg       0.99      0.99      0.99      3000



In [304]:
y_score_rf = rf.predict_proba(X_test_transformed)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_score_rf)
roc_auc = auc(fpr, tpr)

In [305]:
fig_hist = px.histogram(x=y_score_rf, nbins=50, title='Predicted Probabilities of Employee Turnover (Random Forest)', color = y_test.map({0: 'Stayed', 1: 'Left'}), labels={'x': 'Predicted Probability', 'color': 'Actual Turnover'})
fig_hist.show()

In [306]:
df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr
}, index=thresholds)
df.index.name = "Thresholds"
df.columns.name = "Rate"

fig_thresh = px.line(
    df, title='TPR and FPR at every threshold',
    width=700, height=500, render_mode='SVG'
)

fig_thresh.update_yaxes(scaleanchor="x", scaleratio=1)
fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
fig_thresh.show()

gradient boosting

In [307]:
gb = GradientBoostingClassifier()

In [308]:
bg_kfold_scores = cross_val_score(gb, X_train_smote, y_train_smote, cv=skf, scoring='accuracy') 

In [309]:
gb.fit(X_train_smote, y_train_smote)
y_pred_gb = gb.predict(X_test_transformed)
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2284
           1       0.93      0.95      0.94       716

    accuracy                           0.97      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [310]:
y_score_gb = gb.predict_proba(X_test_transformed)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_score_lr)
roc_auc = auc(fpr, tpr)

In [311]:
fig_hist = px.histogram(x=y_score_gb, nbins=50, title='Predicted Probabilities of Employee Turnover (Gradient Boost)', color = y_test.map({0: 'Stayed', 1: 'Left'}), labels={'x': 'Predicted Probability', 'color': 'Actual Turnover'})
fig_hist.show()

In [312]:
df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr
}, index=thresholds)
df.index.name = "Thresholds"
df.columns.name = "Rate"

fig_thresh = px.line(
    df, title='TPR and FPR at every threshold',
    width=700, height=500, render_mode='SVG'
)

fig_thresh.update_yaxes(scaleanchor="x", scaleratio=1)
fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
fig_thresh.show()

based on the classification reports, random forest classifier is the better model

## Clustering Modeling

In [313]:
df = pd.read_csv('HR_comma_sep.csv')
df['Department'] = df['sales']
df = df.drop(columns=['sales'])
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,Department
0,0.38,0.53,2,157,3,0,1,0,low,sales
1,0.80,0.86,5,262,6,0,1,0,medium,sales
2,0.11,0.88,7,272,4,0,1,0,medium,sales
3,0.72,0.87,5,223,5,0,1,0,low,sales
4,0.37,0.52,2,159,3,0,1,0,low,sales


In [314]:
df_left = df[df['left'] == 1]
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,Department
0,0.38,0.53,2,157,3,0,1,0,low,sales
1,0.80,0.86,5,262,6,0,1,0,medium,sales
2,0.11,0.88,7,272,4,0,1,0,medium,sales
3,0.72,0.87,5,223,5,0,1,0,low,sales
4,0.37,0.52,2,159,3,0,1,0,low,sales


In [315]:
fig = px.scatter(df_left, x='satisfaction_level', y='last_evaluation')
fig.show()

three distinct clusters - 1. low satisfaction and high eval score 2. mid satisfaction and low eval score 3. high satisfaction and high eval score

In [316]:
mm.fit_transform(df_left[['satisfaction_level', 'last_evaluation']])

array([[0.34939759, 0.14545455],
       [0.85542169, 0.74545455],
       [0.02409639, 0.78181818],
       ...,
       [0.3373494 , 0.14545455],
       [0.02409639, 0.92727273],
       [0.3373494 , 0.12727273]])

In [317]:
df_left_scaled = pd.DataFrame(mm.fit_transform(df_left[['satisfaction_level', 'last_evaluation']]), columns=['satisfaction_level', 'last_evaluation'])
df_left_scaled.head()

,satisfaction_level,last_evaluation
0,0.3493975904,0.1454545455
1,0.8554216867,0.7454545455
2,0.0240963855,0.7818181818
3,0.7590361446,0.7636363636
4,0.3373493976,0.1272727273


In [318]:
kmeans_3 = KMeans(n_clusters=3)

In [319]:
kmeans_3.fit(df_left_scaled)

KMeans(n_clusters=3)

In [320]:
kmeans_3.cluster_centers_ # centroids of the 3 clusters - scaled values

array([[0.86395634, 0.84770142],
       [0.38748519, 0.12177894],
       [0.02616878, 0.76701632]])

coordinates of the centroids for each cluster

In [321]:
mm.inverse_transform(kmeans_3.cluster_centers_) # coordinates of the centroids for each cluster - actual values

array([[0.80708376, 0.91623578],
       [0.41161271, 0.51697842],
       [0.11172009, 0.87185897]])

In [322]:
print(kmeans_3.labels_)

[1 0 2 ... 1 2 1]


In [323]:
kmeans_3.inertia_

62.92944601364937

sum of squares - distance between the points and the centers

In [324]:
df_left_scaled['Cluster'] = kmeans_3.labels_
df_left_scaled.head()

,satisfaction_level,last_evaluation,Cluster
0,0.3493975904,0.1454545455,1
1,0.8554216867,0.7454545455,0
2,0.0240963855,0.7818181818,2
3,0.7590361446,0.7636363636,0
4,0.3373493976,0.1272727273,1


In [325]:
df_left_scaled['Cluster'].value_counts()

Cluster
1    1668
0     967
2     936
Name: count, dtype: int64

In [326]:
centroids_3 = pd.DataFrame(mm.inverse_transform(kmeans_3.cluster_centers_), columns=['satisfaction_level', 'last_evaluation'])
centroids_3

,satisfaction_level,last_evaluation
0,0.8070837642,0.9162357808
1,0.4116127098,0.5169784173
2,0.1117200855,0.8718589744


In [327]:
fig = px.scatter(df_left, x="satisfaction_level", y="last_evaluation", color=df_left_scaled['Cluster'])
fig.add_trace(go.Scatter(x=centroids_3['satisfaction_level'], y=centroids_3['last_evaluation'], mode='markers', marker=dict(color='red', size=10), name='Centroids', text=centroids_3.index))
fig.update_layout(coloraxis_showscale=False)

d = fig.to_dict()
d["data"][0]["type"] = "scatter"

go.Figure(d)

# elbow curve

In [328]:
int_ = {}
cluster_no = range(1, 30)

for i in cluster_no:
    k_ = KMeans(n_clusters=i).fit(df_left_scaled)
    int_['cluster no_' + str(i)] = k_.inertia_

In [329]:
int_

{'cluster no_1': 2724.8723187853907,
 'cluster no_2': 990.3736058877394,
 'cluster no_3': 62.92944601364937,
 'cluster no_4': 52.19318391026554,
 'cluster no_5': 43.22843130406862,
 'cluster no_6': 35.94714617816901,
 'cluster no_7': 33.042766621677195,
 'cluster no_8': 27.552174123902358,
 'cluster no_9': 25.782843466085406,
 'cluster no_10': 23.207701604606527,
 'cluster no_11': 22.454354955946854,
 'cluster no_12': 18.79873986276119,
 'cluster no_13': 17.68800401817191,
 'cluster no_14': 16.113887780858857,
 'cluster no_15': 14.981093014910869,
 'cluster no_16': 12.376438348699548,
 'cluster no_17': 12.867514770157644,
 'cluster no_18': 10.71987675551591,
 'cluster no_19': 10.069607601405325,
 'cluster no_20': 9.903393020651414,
 'cluster no_21': 9.02660072097093,
 'cluster no_22': 9.414725984614993,
 'cluster no_23': 8.39144970684368,
 'cluster no_24': 7.848219843932055,
 'cluster no_25': 7.393206077432654,
 'cluster no_26': 7.184906279907377,
 'cluster no_27': 7.088506857538819,
 

In [330]:
fig = px.line(x=cluster_no, y=list(int_.values()), title='Elbow Curve', labels={'x':'Number of Clusters', 'y':'Inertia'}, markers=True)
fig.show()

based on the elbow curve, 3 is the best number of clusters (in alignment with the scatterplot)

## Silhouette Score

In [331]:
silhouette_score(df_left_scaled, kmeans_3.fit_predict(df_left_scaled))

np.float64(0.8805525232614273)

used to compare silhouette scores for k means cluster values - this model only has one so no comparison